In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# LCLS Classic model 

In [2]:
from LCLS.bmad import LCLSTaoModel
from LCLS.epics import epics_proxy

import numpy as np
import os
import json
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Make sure this exists
assert 'LCLS_LATTICE' in os.environ

In [4]:
# Cached EPICS pv data
epics = epics_proxy('data/epics_snapshot_2018-03-06T11:22:45.000000-08:00.json', verbose=True)

Loaded data/epics_snapshot_2018-03-06T11:22:45.000000-08:00.json with 1722 PVs


In [5]:
M = LCLSTaoModel('lcls_classic', epics = epics,verbose=True, ploton=False)
print(M)

Initialized Tao with /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmp807oibuc/tao/tao.init
Loading all settings
Reading: settings/klystron_settings.bmad
Written: /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmp807oibuc/tao/settings/klystron_settings.bmad
Reading: settings/linac_settings.bmad
Written: /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmp807oibuc/tao/settings/linac_settings.bmad
Reading: settings/collimator_settings.bmad
Written: /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmp807oibuc/tao/settings/collimator_settings.bmad
Calling: settings/LEM_settings.tao
Written: /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmp807oibuc/tao/settings/LEM_settings.tao
offsetting bunch compressors
call /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmp807oibuc/tao/scripts/BC_offsets.tao
LEMing
call /var/folders/wj/lfgr01993dx79p9cm_skykbw0000gn/T/tmp807oibuc/tao/scripts/LEM.tao
Configured.
_______________________________________________
___________________________________

In [6]:
%%tao 
sho var linac_fudge

-------------------------
Tao> sho var linac_fudge
Variable name:  linac_fudge

 Index  Controlled Attributes(s)    Meas         Model        Design  Useit_opt
     1  O_L1_FUDGE[F]           0.0000E+00    1.0094E+00    1.0000E+00       T
     2  O_L2_FUDGE[F]           0.0000E+00    1.0265E+00    1.0000E+00       T
     3  O_L3_FUDGE[F]           0.0000E+00    1.0366E+00    1.0000E+00       T
 Index  Controlled Attributes(s)    Meas         Model        Design  Useit_opt
-------------------------
Tao> 


## Track particles

In [7]:
%%tao
set beam_init beam_track_end = UEEND
set csr_param n_bin = 40
snparticle 10000
beamon
beamoff

-------------------------
Tao> set beam_init beam_track_end = UEEND
-------------------------
Tao> set csr_param n_bin = 40
-------------------------
Tao> snparticle 10000
-------------------------
Tao> beamon
2470 particle(s) lost at element 280: CE11  Total lost: 2470  of 10000
Beam at Element: 1352. Time: 1 min
Total number of lost particles by the end of universe  1:  2470.
-------------------------
Tao> beamoff
-------------------------
Tao> 


In [8]:
h5file = '/Users/chrisonian/Code/GitHub/lcls2-live/devel/test1.h5'
M.cmd('write beam -at UEBEG '+h5file)

h5file = '/Users/chrisonian/Code/GitHub/lcls2-live/devel/test2.h5'
M.cmd('write beam -at UEEND '+h5file)

['[INFO] tao_write_cmd:',
 '    Written: /Users/chrisonian/Code/GitHub/lcls2-live/devel/test2.h5']

# Plot

In [9]:
from opmd_beamphysics import bunch_plotting, bunch_tools, bunch_stats
from bokeh.plotting import  show, output_notebook
from bokeh.layouts import column, row
output_notebook(verbose=False, hide_banner=True)
from h5py import File

In [17]:
#h1.close()
#h2.close()
h1 = File( '/Users/chrisonian/Code/GitHub/lcls2-live/devel/test1.h5', 'r')
h2 = File( '/Users/chrisonian/Code/GitHub/lcls2-live/devel/test2.h5', 'r')
bunch1 = h1['data/00001/particles']
bunch2 = h2['data/00001/particles']

show(
    row(
        column(    
            bunch_plotting.plot_bunch_h5(bunch1, 't', 'pz', liveOnly=True,  bins = 100),
            bunch_plotting.plot_histogram_h5(bunch1, 't', bins=100)), 
    column(
        
    bunch_plotting.plot_bunch_h5(bunch2, 't', 'pz', liveOnly=True,  bins = 100),
    bunch_plotting.plot_histogram_h5(bunch2, 't', bins=100))        

    )
    
    )